<a href="https://colab.research.google.com/github/DataSpott/SGT-Analysis/blob/master/SGT_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes.\n'.format(name=fn, length=len(uploaded[fn])))

board_count = int(input('Enter the number of checkerboards at your Well-Plate: '))
print('\n')
first_wells_input = input('Enter the first well of each checkerboard (separated by comma): ')
print('\n')
last_wells_input = input('Enter the last well of each checkerboard (separated by comma): ')
print('\n')
log_time = int(input("Enter time (in [min]) that the investigated bacteria needs to grow one log-level (e.g.: 70): "))
print('\n')
antibiotic_one_name_input = input("Enter the name of the first antibiotic (rising in rows, e.g.: Nitroxolin): ")
print('\n')
antibiotic_two_name_input = input("Enter the name of the second antibiotic (rising in columns, e.g.: Dalbavancin): ")
print('\n')
antibiotic_one_input = input("Enter the concentrations of the first antibiotic (rising in rows), in ascending order and separated by commas (e.g.: Nitroxolin, 0, 8, 32, 64): ")
print('\n')
antibiotic_two_input = input("Enter the concentrations of the second antibiotic (rising in columns), in ascending order and separated by commas (e.g.: Dalbavancin, 0, 8, 32, 64): ")

Saving 04.04.2019_Bk14073_Nitro 0-128_Dalba 0-8_Sunrise_rawdata.xlsx to 04.04.2019_Bk14073_Nitro 0-128_Dalba 0-8_Sunrise_rawdata.xlsx
User uploaded file "04.04.2019_Bk14073_Nitro 0-128_Dalba 0-8_Sunrise_rawdata.xlsx" with length 74404 bytes.

Enter the number of checkerboards at your Well-Plate: 2


Enter the first well of each checkerboard (separated by comma): A01, A07


Enter the last well of each checkerboard (separated by comma): G06, G12


Enter time (in [min]) that the investigated bacteria needs to grow one log-level (e.g.: 70): 70


Enter the name of the first antibiotic (rising in rows, e.g.: Nitroxolin): Nitroxolin


Enter the name of the second antibiotic (rising in columns, e.g.: Dalbavancin): Dalbavancin


Enter the concentrations of the first antibiotic (rising in rows), in ascending order and separated by commas (e.g.: Nitroxolin, 0, 8, 32, 64): 0, 4, 8, 16, 32, 64, 128


Enter the concentrations of the second antibiotic (rising in columns), in ascending order and separ

In [4]:
!git clone https://github.com/DataSpott/SGT-Analysis.git
%cd SGT-Analysis/
!python SGT.py board_count log_time antibiotic_one_name_input antibiotic_two_name_input antibiotic_one_input antibiotic_two_input uploaded[0]

Cloning into 'SGT-Analysis'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
^C
[Errno 2] No such file or directory: 'SGT-Analysis/'
/content/SGT-Analysis
  File "SGT.py", line 15
    parser.add_argument('first_well', type = list, help = "First wells of all checkerboards on the plate.")
         ^
SyntaxError: invalid syntax
